# Data-Driven Research Assignment 3: Linked Open Data
This notebook contains the third, collaborative, graded assignment of the 2023 Data-Driven Research course. In this assignment you'll use Linked Open Data tools in order to search for information on the Web in a more thorough way than with Google.

To complete the assignment, complete the highlighted **Part 1, Part 2, Part 3 and Part 4**.

This is a collaborative assignment. In the text cell below, please include all the names of your group members.

If you used code or a solution from the internet (such as StackOverflow) or another external resource, please make reference to it (in any format). Unattributed copied code will be considered plagiarism and therefore fraud.


**Authors of this answer:**

# 1. Introduction

In this exercise, you'll experiment with a very explicit approach to semantics, and experience how powerful a little semantics can be when searching.

You'll use the DBpedia knowledge base, essentially the content of Wikipedia in machine readable form, and explore what explicit semantic enables using [SPARQL](http://en.wikipedia.org/wiki/SPARQL). Queries allowing you to query and search through the Web of Linked Open Data. We will use the Python SPARQLWrapper library to access the DBpedia endpoint.

## 1.1. RDF
Linked Open Data consists of a huge number of small facts, in the form of RDF triples, <*Subject*, *Predicate*, *Object*>, which each consists of a pair of concepts or entities and a relationship between them, such as `Rembrandt birthPlace Leiden`. We work specifically with DBpedia's machine readable information from Wikipedia, in this case http://dbpedia.org/page/Rembrandt:
`dbr:Rembrandt dbo:birthPlace dbr:Leiden`

It's hard to guess upfront how information is encoded in DBpedia, and linked data is all about having unique identifiers for every entity or concept.   The best way is to look at examples, and use google to kickstart to find a particular DBpedia entity.

For example, Google "dbpedia rembrandt" which will give you a neat page with DBpedia facts about him (https://dbpedia.org/page/Rembrandt).   If you look at the link of the "About: Rembrandt" you find the unique link that is the identifier of this entity is http://dbpedia.org/resource/Rembrandt, and entering this link/ID in your browser will generate the overview page. 

Inside DBpedia, you can use a shorthand  `dbr:Rembrandt` (which is defined to unfold to http://dbpedia.org/resource/Rembrandt) as the unique ID, but it also works if you use the long URL!  Recall that DBpedia does not have pages, but only countless RDF triples, and the overview page is just the output of all triples, <`dbr:Rembrandt` *Predicate*, *Object*>, allowing you also to see what further relations and concepts to explore.

## 1.2. SPARQL
SPARQL is the designated query language for RDF modelled on an extended SQL relational database query language.  It uses natural language words like SELECT, DISTINCT, WHERE, ORDER BY, and LIMIT in a very specific way. We introduce it by example, but feel free to backtrack to one of the many tutorial and introductions on the web.  

# 2. Working with SPARQL queries

There is a large database of facts derived from Wikipedia, called [DBpedia](http://dbpedia.org/About), which contains information about everything and the rest.  Let's look at American Films in the first part of this assignment. You will access a so-called SPARQL endpoint for DBpedia through Python. Each film in the category American Films has as a fact about it in DBpedia that is has a relationship with the category American Films, namely, the `dct:subject` property. That is, the film Pulp Fiction has an RDF triple 
`dbr:Pulp_Fiction dct:subject category:American_films`

If you put the name of the entity `Pulp_Fiction`, so `dbr:Pulp_Fiction` which is shorthand for http://dbpedia.org/resource/Pulp_Fiction, your browser will generate a page http://dbpedia.org/page/Pulp_Fiction with a selection of facts  <`dbr:Pulp_Fiction`, ?, ?> in the data base.

Now, let's access this database through Python. We will need the SPARQLWrapper, which does not come with Google Colab by default, so we should install it:

In [ ]:
!pip install sparqlwrapper

Then, we are able to import it and choose the DBPedia endpoint to access that database:

In [ ]:
from SPARQLWrapper import SPARQLWrapper
import pandas as pd
from io import StringIO
from IPython.display import display, HTML

# Specify the DBPedia endpoint
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

# Specify that we want results in the CSV format
sparql.setReturnFormat('csv')

Now, we can try some SPARQL queries. First we set the query as a multi-line string, and then we use the query function to actually run the query.

Note that in the query below, ?film is a variable name, just like a Python variable - we could rename it to anything else, and the result would be the same.

In [90]:
sparql.setQuery("""
    SELECT ?film 
    WHERE {?film dct:subject dbc:2010s_American_films} 
    LIMIT 1000
""")

result = sparql.query().convert().decode("utf-8") 
print(result[:200])

"film"
"http://dbpedia.org/resource/Cabin_Fever:_Patient_Zero"
"http://dbpedia.org/resource/Cabin_Fever_(2016_film)"
"http://dbpedia.org/resource/Caesar_and_Otto's_Deadly_Xmas"
"http://dbpedia.org/res


The result variable now contains a list in CSV format of 1000 links to 2010s American films on dbpedia (we limited the number of results to 1000, but it can be increased). To make it easier to visualize, let's turn it into a Pandas dataframe (this will become more useful as we retrieve more properties):

In [91]:
df = pd.read_csv(StringIO(result), sep=",")
df

,film
0,http://dbpedia.org/resource/Cabin_Fever:_Patie...
1,http://dbpedia.org/resource/Cabin_Fever_(2016_...
2,http://dbpedia.org/resource/Caesar_and_Otto's_...
3,http://dbpedia.org/resource/Café_(2010_film)
4,http://dbpedia.org/resource/Café_Society_(2016...
...,...
995,http://dbpedia.org/resource/Johnny_English_Str...
996,http://dbpedia.org/resource/Johnny_Frank_Garre...
997,http://dbpedia.org/resource/Joint_Body
998,http://dbpedia.org/resource/Jojo_Rabbit


We see that the enitities, e.g. `dbc:American_films` are URI's referring to a unique entity in the linked open data cloud. So the name is a unique ID, in this case shorthand for http://dbpedia.org/resource/Category:2010_American_films.  

Take a closer look at that SPARQL query. Can you figure out how it works? If you're familiar with structured query languages like SQL, you'll recognize many aspects. If this is completely new to you, there are still many recognizable words to help you interpret this query. Let's get back to this later.

To see the power of this form of searching, let's try a slightly more complex query, where you add a second RDF-like condition to the query, separated from the first by a dot `.` representing a `join` or `AND`:

In [ ]:
sparql.setQuery("""
    SELECT ?film ?actor
    WHERE {?film dct:subject dbc:2010s_American_films . ?film dbo:starring ?actor } 
    LIMIT 1000
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df

You should now get a table with American films and the actors that have played in them. This list is not complete. The content of DBpedia is based on the Infoboxes of Wikipedia pages, which have a standard format. The knowledge in DBpedia is as good as the encyclopedic information on Wikipedia. Not all actors per film are listed and not all films and actors have their own Wikipedia article.

Try a few more SPARQL queries, given below. See if you can figure out before hand what results they will give:

In [ ]:
sparql.setQuery("""
    SELECT DISTINCT ?actor
    WHERE {?film dct:subject dbc:2010s_American_films . ?film dbo:starring ?actor } 
    LIMIT 1000
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df

In [ ]:
sparql.setQuery("""
    SELECT DISTINCT COUNT(?actor) ?actor
    WHERE {?film dct:subject dbc:2010s_American_films . ?film dbo:starring ?actor } 
    ORDER BY DESC(COUNT(?actor))
    LIMIT 1000
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df

**Part 1**: Adapt the SPARQL query above to count per film how many actors it has. This requires a minimal change to the query.   

In [ ]:
# Your adapted query here
sparql.setQuery("""
    
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df

# 3. Six Degrees of Kevin Bacon

There is a game related to the notion of [Six Degrees of Separation](http://en.wikipedia.org/wiki/Six_degrees_of_separation). This involves the network of actors who have played in a film together with Kevin Bacon, and actors who have played with those actors, etc. The goal is to figure out the shortest path, between actors who have co-starred in a film, between any Hollywood actor and the actor Kevin Bacon. One of the research aspects is whether the network of actors in Hollywood form a so-called [Small World](http://en.wikipedia.org/wiki/Small-world_experiment) network.

**Part 2** (some questions in the steps below)

Steps:

1. Let's first query:

In [ ]:
sparql.setQuery("""
    SELECT ?film
    WHERE {?film dbo:starring dbr:Kevin_Bacon}
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df

2. How many films are in the list? We can count using a COUNT statement:

In [ ]:
sparql.setQuery("""
    SELECT COUNT(?film)
    WHERE {?film dbo:starring dbr:Kevin_Bacon}
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df

3. Is this list complete? Compare this number with, for instance, the number of films listed on Kevin Bacon's IMDB page.

**(Your answer here)**

4. To see the power of this form of searching, let's gradually make this into a more complex query. Let's ask for the list of actors co-starring with Kevin Bacon. You can add a second RDF-like condition to the query, separated from the first by a dot:

In [ ]:
sparql.setQuery("""
    SELECT ?film ?actor
    WHERE { ?film dbo:starring dbr:Kevin_Bacon . ?film dbo:starring ?actor} 
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df

You should now get a table with films that Kevin Bacon played in, and the actors who played with him in those films. This list also includes Kevin Bacon in each of those films.

5. To remove Kevin Bacon himself from the list of co-actors, you can use a Regular Expression (as discussed in [Coding the Humanities](https://github.com/bloemj/2023-coding-the-humanities/blob/main/notebooks/2_Text.ipynb)) to remove any actor containing the name `Kevin Bacon`, like this:

In [ ]:
sparql.setQuery("""
    SELECT ?film ?actor
    WHERE { ?film dbo:starring dbr:Kevin_Bacon . ?film dbo:starring ?actor .
            FILTER (!regex(?actor, "Kevin_Bacon"))} 
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df

Or even better, using the semantic entity name directly:

In [ ]:
sparql.setQuery("""
    SELECT ?film ?actor
    WHERE { ?film dbo:starring dbr:Kevin_Bacon . ?film dbo:starring ?actor .
            FILTER ( ?actor != dbr:Kevin_Bacon )}
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df

6. To get a list of only the actors, remove `?film` from the selection. Use `SELECT DISTINCT` if needed to avoid duplicates.  Now can you get a count of the number of actors in the list?

In [ ]:
# Your code here (copy from above and adapt)

7. We'll expand the SPARQL query to get actors who co-starred with actors who co-starred with Kevin Bacon, i.e. actors who are two steps away from Kevin Bacon. First get all films that the co-stars of Kevin Bacon played in:

In [ ]:
sparql.setQuery("""
    SELECT DISTINCT ?film1 ?actor1 ?film2 
    WHERE { ?film1 dbo:starring dbr:Kevin_Bacon . ?film1 dbo:starring ?actor1 
               . ?film2 dbo:starring ?actor1 .
               FILTER ( ?actor1 != dbr:Kevin_Bacon )  }
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df

8. Next we add the co-stars of the co-stars of Kevin Bacon as `?actor2`:

In [ ]:
sparql.setQuery("""
    SELECT ?film1 ?actor1 ?film2 ?actor2 
    WHERE { ?film1 dbo:starring dbr:Kevin_Bacon . ?film1 dbo:starring ?actor1 
           . ?film2 dbo:starring ?actor1 . ?film2 dbo:starring ?actor2 .
               FILTER (?actor1 != dbr:Kevin_Bacon && ?actor2 != dbr:Kevin_Bacon ) }
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df

9. How many actors are within two steps of Kevin Bacon?  Are there no duplicates? Note the difference between `COUNT(DISTINCT ?film)` and `COUNT(?film)`.

In [ ]:
# Your code here

**(Your answer here)**

# 4. Wanderlust

DBpedia is great to wander around---just like browsing through Wikipedia---but then with powerful aggregation tools at your finger tips.   Follow this walk, and make your own walks.

## Example Walk

1. Starting is always hard, so let's start with Google "dbpedia rembrandt", as we did in the lecture.

This gives you quite some info, and reveals Rembrandt is a dbpedia resource, hence `dbr:Rembrandt` (shorthand for http://dbpedia.org/resource/Rembrandt) is the unique ID inside dbpedia.

2. Let's see what information there is with `dbr:Rembrandt`

In [ ]:
sparql.setQuery("""
    SELECT ?p ?o WHERE { dbr:Rembrandt ?p ?o }
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df.head(20) #Show the first 30 results

3. The list is partial, but one relation to explore could be the "type" of entity.

In [ ]:
sparql.setQuery("""
    SELECT ?o WHERE { dbr:Rembrandt rdf:type ?o }
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df.head(30) #Show the first 30 results

4. That's a lot---but probably you knew already a lot about him, let's move to another entity that is well-represented in Wikipedia.

In [ ]:
sparql.setQuery("""
    SELECT ?o WHERE { dbr:Darth_Vader rdf:type ?o }
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df.head(30) #Show the first 30 results

5. With so many things, what is his actual occupation?

In [ ]:
sparql.setQuery("""
    SELECT ?occupation WHERE { dbr:Darth_Vader dbo:occupation ?occupation }
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df

6. Wow, but who else then is a Jedi?

In [ ]:
sparql.setQuery("""
    SELECT ?person WHERE { ?person dbo:occupation dbr:Jedi }
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df

7. And who is actually a Sith (spoiler alert)?  

In [ ]:
sparql.setQuery("""
    SELECT ?person WHERE { ?person dbo:occupation dbr:Sith }
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df

8. For those who don't want to know, how many are there?

In [ ]:
sparql.setQuery("""
    SELECT count(?person) WHERE { ?person dbo:occupation dbr:Sith }
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df

9. With Jedi becoming a career opportunity, what occupations are there anyway (incomplete list)?

In [ ]:
sparql.setQuery("""
    SELECT ?person ?occupation WHERE { ?person dbo:occupation ?occupation } ORDER BY ?occupation
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df

10. Impressive, but how many people are there in DBpedia anyway, when looking at nationality?

In [ ]:
sparql.setQuery("""
    SELECT count(?nationality) ?nationality 
       WHERE { ?person dbo:nationality ?nationality } ORDER BY ?nationality
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df

11. Hmm. Some interesting nationalities there... Also this may be a somewhat biased world view---let's sort that by "impact on Wikipedia"?

In [ ]:
sparql.setQuery("""
    SELECT count(?nationality) AS ?count ?nationality 
       WHERE { ?person dbo:nationality ?nationality } ORDER BY DESC(?count)
""")

result = sparql.query().convert().decode("utf-8") 
df = pd.read_csv(StringIO(result), sep=",")
df

12. Etcetera.

**Part 3**

Are there issues with completeness of encoding (does Rembrandt have an occupation or nationality?) or with selection and bias/representation that you observed in this example walk?

**(Your answer here)**

## Make your own walk!

In a similar way as the example walk, make your own walk through DBpedia/Wikipedia. Explore some of the amazing power SPARQL queries give you to explore, as well as being aware of the limitations and bias of the collection and the encoding. We reward creativity as much as technical skills: some of the most interesting queries are a very simple SPARQL statement! Just like in Assignment 1, please do put a topic that you are interested into this one, not one of our boring example ones.

**(Your answers with query code blocks here)**

# 5. Annotating a corpus

Up to now, we have only queried DBpedia/Wikipedia, but the true power of linked open data is the ability to connect any corpus to the entities in DBpedia.  Manually annotating a corpus is very laborious, but automatic tools for entity linking can potentially annotate any DBpedia entity found in any text.

In Python, Spacy has a DBpedia Spotlight-based entity recognizer. Spacy is a very useful Python tool that can handle a large variety of text processing tasks, including also named entity recognition, sentiment analysis, part-of-speech tagging or text categorization, for various languages. It is definitely worth exploring more in your project or some other time.

As this assignment is already long enough, we will only briefly show how it works and ask you to experiment with it in a free format.

Let's install it:

In [ ]:
!pip install spacy-dbpedia-spotlight

We will make a text processing pipeline that only includes the DBedia entity linker, and nothing else:

In [ ]:
import spacy_dbpedia_spotlight
# a new blank model will be created, with the language code provided in the parameter
nlp = spacy_dbpedia_spotlight.create('en')
# in this case, the pipeline will be only contain the EntityLinker
print(nlp.pipe_names)
# ['dbpedia_spotlight']

If you are able to experiment with a different language than English, we encourage you to try it! For that, you can change the 'en' language code above to a different language code.

Now, we simply need to use the nlp function on a string, and it will attempt to recognize DBPedia entities in it:

In [ ]:
doc = nlp('The University Of Amsterdam is a Dutch higher education institution located in Amsterdam.')

print(doc.ents) #This just prints the entities that were found

#This prints some more details, including the DBPedia identifier and the similarity score:
print([(ent.text, ent.kb_id_, ent._.dbpedia_raw_result['@similarityScore']) for ent in doc.ents])

Let's just make a tiny change to the input and see if the output changes:

In [ ]:
doc = nlp('The University of Amsterdam is a Dutch higher education institution located in Amsterdam.')
print([(ent.text, ent.kb_id_, ent._.dbpedia_raw_result['@similarityScore']) for ent in doc.ents])

Interesting. It no longer recognizes the university in my case.

**Part 4**

Now, it's your turn to experiment with entity linking. Find a paragraph of text anywhere in your target language -- text with concrete names of persons, organizations, locations, events, ... (like news) may have more entities than abstract philosophy -- and try it for yourself. How well does this work?  Did it find all or most entities?  Do you see "errors"? 

In [ ]:
# Your experimentation here

**(Your answer here)**

Now assume you have a large text corpus annotated in this way (think up your own corpus of interest, or otherwise think about the corpus of movie reviews you explored earlier).   

Can you think up something you could explore using these annotations? E.g. if we remember the movie reviews collection from Assignment 2 we could look at particular actors, but also about comparing male and female actors as two groups, and doing aggregated queries about male and female actor mentions in the whole corpus of negative and positive movie reviews (similar to the SPARQL queries earlier).

**(Your answer here)**